<a href="https://colab.research.google.com/github/ruohoruotsi/masakhane/blob/master/en-urh/jw300-baseline/English_to_Urhobo_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "urh" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [14]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-urh-baseline


In [15]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 5.1MB/s 


In [16]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-urh.xml.gz not found. The following files are available for downloading:

 304 KB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-urh.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
   3 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/urh.zip

 267 MB Total size
./JW300_latest_xml_en-urh.xml.gz ... 100% of 304 KB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_urh.zip ... 100% of 3 MB


In [17]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2019-12-30 07:21:25--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.02s   

2019-12-30 07:21:25 (12.4 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2019-12-30 07:21:26--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-urh.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

In [18]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [19]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 4050/32709 lines since contained in test set.


,source_sentence,target_sentence
0,Why It Pays to Be Honest 6,Erere Herọ Ra Vwọ Dia Ohwo rẹ Uyota 5
1,The Bible Changes Lives,7 Ovwan “ Jẹn Ẹguọnọ rẹ Iniọvo na Dje Ebuoebuo...
2,Give Me Just One Year of Peace and Happiness 8...,"12 Jẹ ‘ Ẹse rẹ Ọghẹnẹ rẹ Unu se Gbe - e na , ’..."


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [20]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [21]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

     |████████████████████████████████| 51kB 3.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144681 sha256=4537127d21a3fdba6d4bd3bf7e6d305ab4f32d450b996f04a76f46e47d7cf10c
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
00:00:00.04 0.00 percent complete
00:00:21.31 3.74 percent complete
00:00:42.14 7.49 percent complete
00:01:04.35 11.23 percent complete
00:01:26.20 14.98 percent complete
00:01:47.40 18.72 percent complete
00:02:08.42 22.46 percent complete
00:02:29.34 26.21 percent complete
00:02:49.96 29.95 percent complete
00:03:11.20 33.70 percent complete
00:03:32.13 37.44 percent complete
00:03:54.79 41.18 percent complete
00:04:15.89 44.93 percent complete
00:04:36.32 48.67 percent complete
00:04:57.28 52.41 percent complete
00:05:18.66 56.16 percent complete
00:05:40.21 59.90 percent complete


00:06:01.44 63.65 percent complete
00:06:22.21 67.39 percent complete
00:06:45.07 71.13 percent complete
00:07:05.83 74.88 percent complete
00:07:26.54 78.62 percent complete
00:07:47.67 82.37 percent complete
00:08:08.28 86.11 percent complete


00:08:29.51 89.85 percent complete
00:08:50.49 93.60 percent complete
00:09:11.91 97.34 percent complete


In [22]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
The number of publishers is now about ten times what it was when I began serving here .
Similarly , elders should not only encourage and console their brothers with words but also build them up by showing sincere personal interest . ​ — 1 Cor .
17 Why We “ Keep Bearing Much Fruit ”
Now I have a journal that I keep on my desk to schedule upcoming work , and this helps me to schedule myself , not leaving things till the last minute . ”
1 , 2 . ( a ) How do some react to the thought that God has an organization ?
We cannot go to the point of disobeying God or violating our Christian neutrality . ​ — Read 1 Peter 2 : 13 - 17 .
Did this mean freedom for every literal slave ?
Eventually , all my siblings did so and became Jehovah’s Witnesses .
How pleased Jehovah will be as he observes our whole - souled efforts to “ keep bearing much fruit ” !
Joseph , though , was a disciple , but he could not bring himself to say so openly .

==> train.urh <==
Ighwoghwota rehẹ ẹkuotọ na e



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [23]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 2199 (delta 4), reused 5 (delta 3), pack-reused 2184
Receiving objects: 100% (2199/2199), 2.60 MiB | 4.31 MiB/s, done.
Resolving deltas: 100% (1525/1525), done.
Processing /content/joeynmt
     |████████████████████████████████| 266kB 10.9MB/s 
     |████████████████████████████████| 307kB 22.3MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 215kB 25.4MB/s 
     |████████████████████████████████| 737kB 26.4MB/s 
     |████████████████████████████████| 61kB 10.5MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=72136 sha256=77b08ba36545473f912885e7dd1849ba72a600b04598d2c533041a22c4b2334b
  Stored in directory: /tmp/pip-ephem-wheel-cache-irh9ejn6/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyyam

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [0]:
!ls drive/'My Drive'/masakhane/en-urh-baseline/models/enurh_transformer

In [25]:
# ####IOHAVOC -->>  WE DO NOT WANT TO DO BPE BY DEFAULT, we need two different configs, but generate vocab below

# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT 
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 

# Learn BPEs on the training data.
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Urhobo Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.urh    test.urh	    train.en
dev.bpe.en	dev.urh      test.en	     train.bpe.en   train.urh
dev.bpe.urh	test.bpe.en  test.en-any.en  train.bpe.urh
bpe.codes.4000	dev.en	     test.bpe.urh    test.urh	    train.en
dev.bpe.en	dev.urh      test.en	     train.bpe.en   train.urh
dev.bpe.urh	test.bpe.en  test.en-any.en  train.bpe.urh
BPE Urhobo Sentences
Diesorọ H@@ us@@ ha@@ i vwọ guọnọ uduefiogbere ọ sa vwọ fuevun kẹ Ọghẹnẹ ?
Diesorọ ọ vwọ guọnọ uduefiogbere avwanre ke sa fuevun ?
Me nẹrhovwo vwọ kẹ uduefiogbere me sa vwọ yọn@@ regan .
E@@ nẹ@@ na , ẹwẹn rayen kpotọ re , me sa kọn bru ayen ra ọkieje . ” — Se Isẹ 29 : 25 .
[ 1 ] ( ẹkorota 7 ) E wene e@@ dẹ evo .
Combined BPE Vocab
ording
ople
eignty
ẹgbaẹ@@
Heb@@
sider
Babil@@
/@@
rọvw@@
Jihov@@


In [37]:
! ls joeynmt/data/enurh
! ls joeynmt/configs

bpe.codes.4000	dev.urh       test.en-any.en  train.en
dev.bpe.en	test.bpe.en   test.urh	      train.urh
dev.bpe.urh	test.bpe.urh  train.bpe.en    vocab-nonBPE.txt
dev.en		test.en       train.bpe.urh   vocab.txt
iwslt14_deen_bpe.yaml		   transformer_reverse.yaml
iwslt_deen_bahdanau.yaml	   transformer_small.yaml
iwslt_envi_luong.yaml		   transformer_wmt17_ende.yaml
iwslt_envi_xnmt.yaml		   transformer_wmt17_lven.yaml
reverse.yaml			   wmt_ende_best.yaml
small.yaml			   wmt_ende_default.yaml
transformer_copy.yaml		   wmt_lven_best.yaml
transformer_enurh.yaml		   wmt_lven_default.yaml
transformer_iwslt14_deen_bpe.yaml


In [0]:
# ####IOHAVOC CREATE THE VOCAB FOR NON-BPE EXPERIMENTS
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.$src joeynmt/data/$src$tgt/train.$tgt --output_path joeynmt/data/$src$tgt/vocab-nonBPE.txt


In [36]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.en	     test.bpe.urh    test.urh	    train.en
dev.bpe.en	dev.urh      test.en	     train.bpe.en   train.urh
dev.bpe.urh	test.bpe.en  test.en-any.en  train.bpe.urh


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 150                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000           # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: True                # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [55]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2019-12-30 08:41:49,260 Hello! This is Joey-NMT.
2019-12-30 08:41:50,622 Total params: 12119552
2019-12-30 08:41:50,623 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [56]:
# Copy the created models from the notebook storage to google drive for persistant storage 
#!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"
!cp -r joeynmt/models/${src}${tgt}_transformer/* drive/'My Drive'/masakhane/en-urh-baseline/models/enurh_transformer

cp: cannot create symbolic link 'drive/My Drive/masakhane/en-urh-baseline/models/enurh_transformer/best.ckpt': Operation not supported


In [57]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 1000	Loss: 82894.25781	PPL: 33.57090	bleu: 1.38076	LR: 0.00030000	*
Steps: 2000	Loss: 71127.99219	PPL: 20.38746	bleu: 3.38906	LR: 0.00030000	*
Steps: 3000	Loss: 64387.64844	PPL: 15.32086	bleu: 5.88120	LR: 0.00030000	*
Steps: 4000	Loss: 60269.48047	PPL: 12.86689	bleu: 7.95364	LR: 0.00030000	*
Steps: 5000	Loss: 57490.78516	PPL: 11.43726	bleu: 9.62943	LR: 0.00030000	*
Steps: 6000	Loss: 55914.19141	PPL: 10.69792	bleu: 10.53420	LR: 0.00030000	*
Steps: 7000	Loss: 54344.22656	PPL: 10.00918	bleu: 11.26628	LR: 0.00030000	*
Steps: 8000	Loss: 53979.01953	PPL: 9.85543	bleu: 11.76212	LR: 0.00030000	*
Steps: 9000	Loss: 52996.83594	PPL: 9.45355	bleu: 12.13361	LR: 0.00030000	*
Steps: 10000	Loss: 52344.57031	PPL: 9.19576	bleu: 12.86276	LR: 0.00030000	*
Steps: 11000	Loss: 52315.73438	PPL: 9.18453	bleu: 13.28933	LR: 0.00030000	*
Steps: 12000	Loss: 51660.80469	PPL: 8.93306	bleu: 13.25257	LR: 0.00030000	*
Steps: 13000	Loss: 51232.82812	PPL: 8.77247	bleu: 13.75931	LR: 0.00030000	*
Steps: 14000	Loss: 

In [58]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2019-12-30 13:49:27,762 Hello! This is Joey-NMT.
2019-12-30 13:49:49,656  dev bleu:  15.91 [Beam search decoding with beam size = 5 and alpha = 1.0]
2019-12-30 13:50:27,825 test bleu:  28.82 [Beam search decoding with beam size = 5 and alpha = 1.0]
